# Perform multi-channel convolution 
mainly focusing on the implementation of convolution in convolutional neural network iwth forward pass and backward pass

In [1]:
import numpy as np

# 2D convolution: forward and backward

In [2]:
def conv_2d(x, k):
    """
    Perform 2-D convolution
    :param x: input image
    :param k: kernel
    :return: the result of 2-D convolution
    """
    H, W = x.shape
    KH, KW = k.shape

    H_out = H - KH + 1
    W_out = W - KW + 1

    # Convert x into a matrix using indices
    i0 = np.repeat(np.arange(KH), KW)
    i1 = np.repeat(np.arange(H_out), W_out)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    # print(i)

    j0 = np.tile(np.arange(KW), KH)
    j1 = np.tile(np.arange(W_out), H_out)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    # print(j)

    x_crop = x[i, j]

    ## flatten kernel 
    k_crop = k.flatten()

    output = k_crop.dot(x_crop).reshape([H_out, W_out])

    return output

2-D convolution back-propagation

In [4]:
def back_propagation_2d(x,k,grad):
    x_grad = np.ones_like(x)
    k_grad = np.ones_like(k)
    k_grad = conv_2d(x, grad)
    g_padded = np.pad(grad, ((1, 1), (1, 1)), mode='constant')
    k_flip = np.flip(k, 1)
    k_flip = np.flip(k_flip, 0)
    x_grad = conv_2d(g_padded,k_flip)
    return x_grad, k_grad

In [5]:
x1 = np.arange(1, 17, 1).reshape([4, 4])
k1 = np.arange(1, 5, 1).reshape([2, 2])

res_conv2d = conv_2d(x1, k1)
# grad = np.arange(1,10,1).reshape(res_conv2d.shape)
grad1 = np.ones_like(res_conv2d)
x_grad, k_grad = back_propagation_2d(x1,k1,grad1)
print(x_grad)
print(k_grad)

[[ 1  3  3  2]
 [ 4 10 10  6]
 [ 4 10 10  6]
 [ 3  7  7  4]]
[[54 63]
 [90 99]]


# 3-D convolution: forward and backward pass

In [7]:
def get_indices(x,k):
    B, C1, H, W = x.shape
    C2, _, KH, KW = k.shape

    H_out = H - KH + 1
    W_out = W - KW + 1

    i0 = np.repeat(np.arange(KH), KW)
    i0 = np.tile(i0, C1)
    i1 = np.repeat(np.arange(H_out), W_out)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)

    j0 = np.tile(np.arange(KW), KH * C1)
    j1 = np.tile(np.arange(W_out), H_out)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    m = np.repeat(np.arange(C1), KH * KW).reshape(-1, 1)
    
    return m,i,j

In [8]:
def conv_3d(x, k):
    """
    Perform 3-D convolution
    :param x: input image with size ([B,C1,H,W]). Otherwise, use np.transpose to reconstruct the shape as formulated.
    :param k: kernel with size ([C2,C1,KH,KW]). Otherwise, use np.transpose to reconstruct the shape as formulated.
    :return: the result of 3-D convolution in convolutional neural network.
    """
    B, C1, H, W = x.shape
    C2, _, KH, KW = k.shape

    H_out = H - KH + 1
    W_out = W - KW + 1

    m,i,j = get_indices(x,k)

    x_crop = x[:, m, i, j]
    k_crop = k.reshape(C2, -1)

    output = k_crop.dot(x_crop).transpose(1, 0, 2)
    return output.reshape([B, C2, H_out, W_out])

In [15]:
x2 = np.arange(1, 97, 1).reshape([2, 3, 4, 4])
k2 = np.arange(1, 25, 1).reshape([2, 3, 2, 2])

res = conv_3d(x2, k2)
res

array([[[[ 2060,  2138,  2216],
         [ 2372,  2450,  2528],
         [ 2684,  2762,  2840]],

        [[ 4868,  5090,  5312],
         [ 5756,  5978,  6200],
         [ 6644,  6866,  7088]]],


       [[[ 5804,  5882,  5960],
         [ 6116,  6194,  6272],
         [ 6428,  6506,  6584]],

        [[15524, 15746, 15968],
         [16412, 16634, 16856],
         [17300, 17522, 17744]]]])

3-D back-propagation with looping over the kernel 

In [10]:
def back_propagation_baseline(x, k, grad):
    x_grad = np.zeros_like(x)
    k_grad = np.zeros_like(k)
    B, H, W, C1 = x.shape
    KH, KW, _, C2 = k.shape
    Ho = H - KH + 1
    Wo = W - KW + 1
    ygrad = np.reshape(grad, [-1, C2])

    for i in range(KH):
        for j in range(KW):
            xij = np.matmul(ygrad, k[i, j, :, :].T)
            xij = np.reshape(xij, [B, Ho, Wo, C1])
            x_grad[:, i:(Ho + i), j:(Wo + j), :] = x_grad[:, i:(Ho + i),
                                                   j:(Wo + j), :] + xij
            kij = x[:, i:(Ho + i), j:(Wo + j), :]
            kij = np.reshape(kij, [-1, C1])
            k_grad[i, j, :, :] = k_grad[i, j, :, :] + np.matmul(kij.T, ygrad)

    return x_grad, k_grad

3-D back-propagation using fancy indices

In [29]:
def back_propagation_3d(x, k, grad):
    x_grad = np.zeros_like(x)
    k_grad = np.zeros_like(k)
    
    B, C1, H, W = x.shape
    C2, _, KH, KW = k.shape
    H_out = H - KH + 1
    W_out = W - KW + 1

    m,i,j = get_indices(x,k)
    
    x_crop = x[:,m,i,j].transpose(1,2,0).reshape([KH*KW*C1,-1])
    y_grad = grad.transpose(1,2,3,0).reshape([C2,-1])
    k_grad = y_grad.dot(x_crop.T).reshape(k.shape)
    
    dx = k.reshape([C2,-1]).T.dot(y_grad)
    dx_reshaped =  dx.reshape([C1*KH*KW,-1,B]).transpose(2,0,1)
    np.add.at(x_grad, (slice(None),m,i,j), dx_reshaped)
    
    return x_grad, k_grad

In [30]:
x3 = x2.transpose(0, 2, 3, 1)
k3 = k2.transpose(2, 3, 1, 0)
grad = np.ones_like(res).transpose(0, 2, 3, 1)

# print(x3.shape, k3.shape, grad.shape)
x_grad_baseline, k_grad_baseline = back_propagation_baseline(x3, k3, grad)
print(x_grad_baseline.transpose(0,3,1,2))
print('*' * 50)
print(k_grad_baseline.transpose(3,2,0,1))

[[[[ 14  30  30  16]
   [ 32  68  68  36]
   [ 32  68  68  36]
   [ 18  38  38  20]]

  [[ 22  46  46  24]
   [ 48 100 100  52]
   [ 48 100 100  52]
   [ 26  54  54  28]]

  [[ 30  62  62  32]
   [ 64 132 132  68]
   [ 64 132 132  68]
   [ 34  70  70  36]]]


 [[[ 14  30  30  16]
   [ 32  68  68  36]
   [ 32  68  68  36]
   [ 18  38  38  20]]

  [[ 22  46  46  24]
   [ 48 100 100  52]
   [ 48 100 100  52]
   [ 26  54  54  28]]

  [[ 30  62  62  32]
   [ 64 132 132  68]
   [ 64 132 132  68]
   [ 34  70  70  36]]]]
**************************************************
[[[[ 540  558]
   [ 612  630]]

  [[ 828  846]
   [ 900  918]]

  [[1116 1134]
   [1188 1206]]]


 [[[ 540  558]
   [ 612  630]]

  [[ 828  846]
   [ 900  918]]

  [[1116 1134]
   [1188 1206]]]]


In [31]:
x4 = x2
k4 = k2
grad4 = np.ones_like(res)

x_grad, k_grad = back_propagation_3d(x4, k4, grad4)

print(x_grad)
print('*' * 50)
print(k_grad)

[[[[ 14  30  30  16]
   [ 32  68  68  36]
   [ 32  68  68  36]
   [ 18  38  38  20]]

  [[ 22  46  46  24]
   [ 48 100 100  52]
   [ 48 100 100  52]
   [ 26  54  54  28]]

  [[ 30  62  62  32]
   [ 64 132 132  68]
   [ 64 132 132  68]
   [ 34  70  70  36]]]


 [[[ 14  30  30  16]
   [ 32  68  68  36]
   [ 32  68  68  36]
   [ 18  38  38  20]]

  [[ 22  46  46  24]
   [ 48 100 100  52]
   [ 48 100 100  52]
   [ 26  54  54  28]]

  [[ 30  62  62  32]
   [ 64 132 132  68]
   [ 64 132 132  68]
   [ 34  70  70  36]]]]
**************************************************
[[[[ 540  558]
   [ 612  630]]

  [[ 828  846]
   [ 900  918]]

  [[1116 1134]
   [1188 1206]]]


 [[[ 540  558]
   [ 612  630]]

  [[ 828  846]
   [ 900  918]]

  [[1116 1134]
   [1188 1206]]]]


In [32]:
x_grad.all()==x_grad_baseline.all()

True

In [33]:
k_grad.all()==k_grad_baseline.all()

True